In [1]:
# https://blog.devgenius.io/how-to-build-a-scraping-tool-for-linkedin-in-7-minutes-tool-data-science-csv-selenium-beautifulsoup-python-a673f12ac579
# !pip install selenium

In [1]:
# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
job_name = input("Enter the job name: ")

Enter the job name: Data Scientist


In [3]:
country_name =input("Enter the job location: ")

Enter the job location: United States


In [4]:
# job_name = "Data Scientist"
# country_name = "United States"

job_url ="";
for item in job_name.split(" "):
    if item != job_name.split(" ")[-1]:
        job_url = job_url + item + "%20"
    else:
        job_url = job_url + item

country_url ="";
for item in country_name.split(" "):
    if item != country_name.split(" ")[-1]:
        country_url = country_url + item + "%20"
    else:
        country_url = country_url + item

url = "https://www.linkedin.com/jobs/search?keywords={0}&location={1}&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0".format(job_url,country_url)

In [3]:
# import time
# from selenium import webdriver
# def main():
#     a = webdriver.Chrome()
#     a.get('https://www.google.com')
#     time.sleep(5)
#     a.quit()
# main()

In [5]:
# Creating a webdriver instance
driver = webdriver.Chrome("ChromeDriver_Path/chromedriver")
# Opening the url we have just defined in our browser
driver.get(url)

In [6]:
#We find how many jobs are offered.
jobs_num = driver.find_element(By.CSS_SELECTOR,"h1>span").get_attribute("innerText")
if len(jobs_num.split(',')) > 1:
    jobs_num = int(jobs_num.split(',')[0])*1000
else:
    jobs_num = int(jobs_num)

jobs_num   = int(jobs_num)
print(jobs_num)

335000


In [7]:
numbers = input("Enter the numbers of jobs needed: ")

Enter the numbers of jobs needed: 100


In [24]:
#We create a while loop to browse all jobs. 
numbers = int(numbers)
i = 2
while i <= int(numbers/2)+1:
    #We keep scrollind down to the end of the view.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    print("Current at: ", i, "Percentage at: ", ((i+1)/(int(numbers/2)+1))*100, "%",end="\r")
    try:
        #We try to click on the load more results buttons in case it is already displayed.
        infinite_scroller_button = driver.find_element(By.XPATH, ".//button[@aria-label='Load more results']")
        infinite_scroller_button.click()
        time.sleep(0.1)
    except:
        #If there is no button, there will be an error, so we keep scrolling down.
        time.sleep(0.1)
        pass

In [25]:
#We get a list containing all jobs that we have found.
job_lists = driver.find_element(By.CLASS_NAME,"jobs-search__results-list")
jobs = job_lists.find_elements(By.TAG_NAME,"li") # return a list

#We declare void list to keep track of all obtaind data.
job_title_list = []
company_name_list = []
location_list = []
date_list = []
job_link_list = []

#We loof over every job and obtain all the wanted info.
for job in jobs:
    #job_title
    job_title = job.find_element(By.CSS_SELECTOR,"h3").get_attribute("innerText")
    job_title_list.append(job_title)
    
    #company_name
    company_name = job.find_element(By.CSS_SELECTOR,"h4").get_attribute("innerText")
    company_name_list.append(company_name)
    
    #location
    location = job.find_element(By.CSS_SELECTOR,"div>div>span").get_attribute("innerText")
    location_list.append(location)
    
    #date
    date = job.find_element(By.CSS_SELECTOR,"div>div>time").get_attribute("datetime")
    date_list.append(date)
    
    #job_link
    job_link = job.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
    job_link_list.append(job_link)

jobs_df = pd.DataFrame({'Job Title': job_title_list,
              'Company': company_name_list,
              'Location': location_list,
              'Date': date_list,
              'Link': job_link_list
            })
jobs_df

,Job Title,Company,Location,Date,Link
0,"Data Scientist (Deep Learning), Peacock",Peacock,"New York, NY",2022-12-06,https://www.linkedin.com/jobs/view/data-scient...
1,Data Scientist,Amicus,"New York, NY",2022-12-06,https://www.linkedin.com/jobs/view/data-scient...
2,Jr. Data Scientist,UMortgage,Greater Philadelphia,2022-12-09,https://www.linkedin.com/jobs/view/jr-data-sci...
3,Data Scientist,Pebble,"Seattle, WA",2022-12-15,https://www.linkedin.com/jobs/view/data-scient...
4,Data Scientist,Walmart,"Sunnyvale, CA",2022-12-15,https://www.linkedin.com/jobs/view/data-scient...
...,...,...,...,...,...
270,Machine Learning Engineer,Pfizer,"Peapack - Gladstone, NJ",2022-12-13,https://www.linkedin.com/jobs/view/machine-lea...
271,Machine Learning Engineer,BioSpace,"Pasadena, CA",2022-12-12,https://www.linkedin.com/jobs/view/machine-lea...
272,Machine Learning Engineer,Insight Global,"Denver, CO",2022-12-07,https://www.linkedin.com/jobs/view/machine-lea...
273,Data Scientist,FDA,"Silver Spring, MD",2022-12-15,https://www.linkedin.com/jobs/view/data-scient...


In [27]:
import os
jobs_df.to_csv('Sample_data.csv')